In [1]:
import os
import cv2
import glob
import time
import pickle
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
import random
from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [11]:
def save_p(data, name):
    with open(name, 'wb') as f:
        pickle.dump(data, f)
def load_p(name):
    with open(name, 'rb') as f:
        data = pickle.load(f)
    return data

In [12]:
def plot(img, converted, title1='Original', title2='Converted', cmap="gray", flag1=False, flag2=False):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    f.subplots_adjust(hspace = .2, wspace=.05)
    ax1.imshow(img)
    ax1.set_title(title1, fontsize=30)
    ax2.imshow(converted, cmap=cmap)
    ax2.set_title(title2, fontsize=30)
    
    if flag1 == True:
        mpimg.imsave("output_images/" + title1, img, format="jpg")
    if flag2 == True:
        mpimg.imsave("output_images/" + title2, converted, format="jpg")

## functions for features extraction

In [4]:
def grayscale(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray
    
def hsvscale(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    return hsv

def hlsscale(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    return hls

def luvscale(img):
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    return luv

def labscale(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    return lab

def yuvscale(img):
    yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    return yuv

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [13]:
def color_space(img, flag="gray"):
    if flag=="gray":
        res=grayscale(img)
    elif flag=="hsv":
        res=grayscale(img)
    elif flag=="hls":
        res=grayscale(img)
    elif flag=="luv":
        res=grayscale(img)
    elif flag=="lab":
        res=grayscale(img)
    elif flag=="yuv":
        res=grayscale(img)
    return res

In [5]:
def spatial_features(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

In [6]:
def color_hist(img, nbins=32, bins_range=(0, 256), flag=False):
    rh = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    gh = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    bh = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    bin_edges = rh[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    features = np.concatenate((rh[0], gh[0], bh[0]))
    
    if flag:
        fig = plt.figure(figsize=(12,3))
        plt.subplot(131)
        plt.bar(bin_centers, rh[0])
        #plt.bar(bin_centers, rhist[0])
        plt.xlim(0, 256)
        plt.title('R Histogram')
        plt.subplot(132)
        plt.bar(bin_centers, gh[0])
        plt.xlim(0, 256)
        plt.title('G Histogram')
        plt.subplot(133)
        plt.bar(bin_centers, bh[0])
        plt.xlim(0, 256)
        plt.title('B Histogram')
        
    return rh[0], gh[0], bh[0], bin_centers, features

In [36]:
def f_hog(channel, pix_per_cell=16, cell_per_block=2, orient=11, flag=False):
    
    if flag:
        features, hog_image = hog(channel, orientations=orient,
                              pixels_per_cell=(pix_per_cell, pix_per_cell), 
                              cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=True, visualise=flag, feature_vector=False,
                                  block_norm="L2-Hys")
        return hog_image, features
    else:
        features = hog(channel, orientations=orient,
                      pixels_per_cell=(pix_per_cell, pix_per_cell), 
                      cells_per_block=(cell_per_block, cell_per_block),
                          transform_sqrt=True, visualise=flag, feature_vector=False,
                          block_norm="L2-Hys")
        return features

## feature for classifier

In [80]:
def routine_features(X, cars):
    for car in cars:
        #car=gaussian_blur(car, 5)
        car=choose_color_space(car, "yuv")
        channels=[car[:,:,0], car[:,:,1], car[:,:,2]]
        features=[]
        for c in channels:
            f = f_hog(c)
            f=f.ravel()
            features.extend(f)
        X.append(features)
    return X

def compute_features(vehicles, non_vehicles):
    X = []
    X=routine_features(X, vehicles)
    X=routine_features(X, non_vehicles)
    X=np.array(X)
    return X

In [81]:
# def combine_features(img):
#     gray=grayscale(img)
#     rh, gh, bh, bin_centers, col_features=color_hist(img)
#     hog_image, hog_features = f_hog(gray)
#     features = np.vstack(feature_list).astype(np.float64)
#     return features

## load training dataset

In [84]:
def routine(l, gti):
    for name in gti:
        img=cv2.imread(name)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        l.append(img)
    return l
def load_train_dataset():
    vehicles=[]
    non_vehicles=[]
    
    GTI=glob.glob("./non-vehicles/non-vehicles/GTI/*.png")
    Extra=glob.glob("./non-vehicles/non-vehicles/Extras/*.png")
    
    non_vehicles=routine(non_vehicles, GTI)
    non_vehicles=routine(non_vehicles, Extra)
    
    GTI_far=glob.glob("./vehicles/vehicles/GTI_Far/*.png")
    GTI_Left=glob.glob("./vehicles/vehicles/GTI_Left/*.png")
    GTI_MiddleClose=glob.glob("./vehicles/vehicles/GTI_MiddleClose/*.png")
    GTI_Right=glob.glob("./vehicles/vehicles/GTI_Right/*.png")
    KITTI=glob.glob("./vehicles/vehicles/KITTI_extracted/*.png")
    
    vehicles=routine(vehicles, GTI_far)
    vehicles=routine(vehicles, GTI_Left)
    vehicles=routine(vehicles, GTI_MiddleClose)
    vehicles=routine(vehicles, GTI_Right)
    vehicles=routine(vehicles, KITTI)
    
    return vehicles, non_vehicles

In [88]:
vehicles, non_vehicles=load_train_dataset()

In [89]:
X=compute_features(vehicles, non_vehicles)

In [90]:
X.shape

(17760, 1188)

In [91]:
Y = np.hstack([np.ones(len(vehicles)), np.zeros(len(non_vehicles))])

In [53]:
Y.shape

(17760,)

In [93]:
data={"X":X, "Y":Y}

In [94]:
#save_p(data, "data.pickle")

In [37]:
data=load_p("./data.pickle")

In [38]:
X = data["X"]
Y = data["Y"]

In [39]:
X.shape

(17760, 1188)

# classifier

In [44]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [45]:
for i in range(20):
    state = np.random.randint(0, 100)
    x_train, x_cv, y_train, y_cv = train_test_split(X_scaled, Y, test_size=0.2, random_state=state, stratify=Y)

In [46]:
clf=svm.SVC(kernel='rbf')

In [47]:
clf.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [48]:
clf.score(x_train, y_train)

0.99311811811811812

In [49]:
clf.score(x_cv, y_cv)

0.99099099099099097

In [54]:
data["clf"] = clf
data["scaler"] = scaler

In [55]:
#save_p(data, "data.pickle")

In [11]:
classifiers = {"svm_rbf":svm.SVC(kernel='rbf')}
parameters = {"svm_rbf":{"C" : [0.1, 0.5, 1, 5, 10], "gamma" : [0.1, 0.5, 1, 3]}}

In [12]:
def grid_routine(clf,parameters):
    
    grid = GridSearchCV(clf, parameters, scoring="accuracy")
    grid.fit(x_train,y_train)
    clf = grid.best_estimator_ 
    train_score = clf.score(x_train,y_train)
    cv_score = cross_val_score(clf, x_cv, y_cv, cv=3, scoring = "accuracy").mean()
    
    return clf, train_score, cv_score

In [ ]:
start=time.time()
d = []
for c in ["svm_rbf"]:
    clf=classifiers[c]
    par=parameters[c]
    clf_best, train_score , cv_score = grid_routine(clf, par)
    d[c] = [clf_best, train_score , cv_score]
end=time.time()
print(end-start)